### Step 1: Import and Install Dependencies

In [4]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp


### Step 2: Keypoints using MP Holistic

In [7]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [8]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR-CONVERSION BGR-to-RGB
    image.flags.writeable = False                  # Convert image to not-writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Convert image to writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR-COVERSION RGB-to-BGR
    return image, results

In [9]:
def draw_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                                mp_drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
                                mp_drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1)
                                ) 
    
    # Draw pose connections 
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(80, 22, 10), thickness=2, circle_radius=4),
                                mp_drawing.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2)
                                )

    # Draw left hand connections 
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2)
                                ) 

    # Draw right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                ) 

In [10]:
# cap = cv2.VideoCapture(0)

# # Set mediapipe model
# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
#     while cap.isOpened():
#         # Read feed
#         ret, frame = cap.read()

#         # Flip the frame horizontally
#         # flipped_frame = cv2.flip(frame, 1)

#         # Make detections
#         image, results = mediapipe_detection(frame, holistic)
#         print(results)
        
#         # Draw landmarks
#         draw_landmarks(image, results)

#         # Show to screen
#         cv2.imshow('FluentHands Feed', image)

#         # Break Gracefully
#         if cv2.waitKey(10) & 0xFF == ord('q'):
#             break

#     cap.release()
#     cv2.destroyAllWindows()

In [11]:
# draw_landmarks(frame, results)

In [12]:
# plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

### Step 3: Extract Keypoint Values

In [13]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [14]:
# result_test = extract_keypoints(results)
# len(result_test)

In [15]:
# np.save('0', result_test)
# np.load('0.npy')

### Step 4: Setup Folders for Collection

In [16]:
import json

# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

with open('labels.json', 'r') as file:
    # Actions that we try to detect
    actions = np.array(json.load(file))

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

In [17]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

### Step 5: Collect Keypoint Values for Training and Testing

In [20]:
cap = cv2.VideoCapture(0)

# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
                print(results)
                
                # Draw landmarks
                draw_landmarks(image, results)

                # Wait Logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    # cv2.imshow('FluentHands Feed', image)
                    cv2.waitKey(1000) #i wanna make this longer
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                
                # Export Keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Show to screen
                cv2.imshow('FluentHands Feed', image)

                # Break Gracefully
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

: 

### Step 6: Preprocess Data and Create Labels and Features

In [13]:
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [22]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [23]:
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'A': 0,
 'B': 1,
 'C': 2,
 'D': 3,
 'E': 4,
 'F': 5,
 'G': 6,
 'H': 7,
 'I': 8,
 'J': 9,
 'K': 10,
 'L': 11,
 'M': 12,
 'N': 13,
 'O': 14,
 'P': 15,
 'Q': 16,
 'R': 17,
 'S': 18,
 'T': 19,
 'U': 20,
 'V': 21,
 'W': 22,
 'X': 23,
 'Y': 24,
 'Z': 25,
 'Hello': 26,
 'Thank You': 27}

In [27]:
sequences, labels = [], []

for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])
    # for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
    #     window = []
    #     for frame_num in range(sequence_length):
    #         res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
    #         window.append(res)
    #     sequences.append(window)
    #     labels.append(label_map[action])

In [28]:
np.array(sequences).shape

(840, 30, 1662)

In [29]:
np.array(labels).shape

(840,)

In [30]:
X = np.array(sequences)

In [31]:
X.shape

(840, 30, 1662)

In [32]:
y = to_categorical(labels).astype(int)
y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [34]:
y_train.shape

(798, 28)

In [35]:
len(X_test)

42

### Step 7: Build and Train LSTM Neural Network

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [5]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [18]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [19]:
model.compile(optimizer='Adamax', loss='categorical_crossentropy', metrics=['accuracy'])

In [54]:
# model.fit(X_train, y_train, epochs=284, callbacks=[tb_callback])
model.fit(X_train, y_train, epochs=200, callbacks=[tb_callback])

Epoch 1/200
25/25 [==============================] - 6s 68ms/step - loss: 5.1607 - accuracy: 0.0376
Epoch 2/200
25/25 [==============================] - 2s 68ms/step - loss: 3.3390 - accuracy: 0.0414
Epoch 3/200
25/25 [==============================] - 2s 67ms/step - loss: 3.2996 - accuracy: 0.0351
Epoch 4/200
25/25 [==============================] - 2s 68ms/step - loss: 3.2563 - accuracy: 0.0363
Epoch 5/200
25/25 [==============================] - 2s 68ms/step - loss: 3.1816 - accuracy: 0.0702
Epoch 6/200
25/25 [==============================] - 2s 66ms/step - loss: 3.1424 - accuracy: 0.0965
Epoch 7/200
25/25 [==============================] - 2s 67ms/step - loss: 3.1000 - accuracy: 0.1165
Epoch 8/200
25/25 [==============================] - 2s 67ms/step - loss: 3.0115 - accuracy: 0.1140
Epoch 9/200
25/25 [==============================] - 2s 71ms/step - loss: 2.9256 - accuracy: 0.1717
Epoch 10/200
25/25 [==============================] - 2s 71ms/step - loss: 2.8464 - accuracy: 0.1930

In [29]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 28)                924       
                                                                 
Total params: 597500 (2.28 MB)
Trainable params: 597500 

### Step 8: Make Predictions

In [37]:
res = model.predict(X_test)

2/2 [==============================] - 0s 11ms/step


In [38]:
actions[np.argmax(res[5])]

'H'

In [39]:
actions[np.argmax(y_test[5])]

'H'

### Step 9: Save Weights

In [ ]:
import pickle
pickle.dump(model, open('model.pkl', 'wb'))

In [37]:
# model.save('action.json')
model.save('action_new.h5')

c:\Users\banan\Desktop\FluentHands\FluentHands\.venv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [36]:
model.load_weights('action_new.h5')

### Step 10: Evaluation using Confusion Matrix and Accuracy

In [45]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [40]:
yhat = model.predict(X_test)

2/2 [==============================] - 0s 10ms/step


In [41]:

ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [42]:
yhat

[16,
 22,
 22,
 0,
 22,
 7,
 3,
 22,
 11,
 16,
 24,
 25,
 7,
 5,
 1,
 19,
 10,
 7,
 21,
 4,
 14,
 23,
 21,
 7,
 4,
 23,
 3,
 25,
 1,
 24,
 12,
 10,
 7,
 19,
 15,
 22,
 18,
 0,
 22,
 25,
 3,
 19]

In [43]:
ytrue

[16,
 22,
 22,
 0,
 22,
 7,
 3,
 22,
 11,
 16,
 24,
 25,
 7,
 5,
 1,
 19,
 10,
 7,
 21,
 4,
 14,
 23,
 20,
 7,
 4,
 23,
 3,
 25,
 1,
 24,
 12,
 10,
 7,
 18,
 15,
 22,
 18,
 0,
 22,
 25,
 3,
 19]

In [46]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[40,  0],
        [ 0,  2]],

       [[40,  0],
        [ 0,  2]],

       [[39,  0],
        [ 0,  3]],

       [[40,  0],
        [ 0,  2]],

       [[41,  0],
        [ 0,  1]],

       [[37,  0],
        [ 0,  5]],

       [[40,  0],
        [ 0,  2]],

       [[41,  0],
        [ 0,  1]],

       [[41,  0],
        [ 0,  1]],

       [[41,  0],
        [ 0,  1]],

       [[41,  0],
        [ 0,  1]],

       [[40,  0],
        [ 0,  2]],

       [[40,  0],
        [ 1,  1]],

       [[39,  1],
        [ 0,  2]],

       [[41,  0],
        [ 1,  0]],

       [[40,  1],
        [ 0,  1]],

       [[36,  0],
        [ 0,  6]],

       [[40,  0],
        [ 0,  2]],

       [[40,  0],
        [ 0,  2]],

       [[39,  0],
        [ 0,  3]]], dtype=int64)

In [47]:
accuracy_score(ytrue, yhat)

0.9523809523809523

### Step 11: Test in Real Time

In [38]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [39]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.7

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        # print(results)
        
        # Draw landmarks
        draw_landmarks(image, results)
        
        # Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print('sequence: ', sequence)
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
            # Visualization logic
            # if np.unique(predictions[-10:])[0]==np.argmax(res): 
            #     if res[np.argmax(res)] > threshold: 
                    
            #         if len(sentence) > 0: 
            #             if actions[np.argmax(res)] != sentence[-1]:
            #                 sentence.append(actions[np.argmax(res)])
            #         else:
            #             sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Visualization probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('FluentHands Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        
    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 25ms/step
sequence:  [array([ 0.35630441,  0.45987695, -0.86650926, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.35720187,  0.45968679, -0.79615319, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.35755199,  0.45946664, -0.78414339, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.35915434,  0.45961809, -0.7320441 , ...,  0.        ,
        0.        ,  0.        ]), array([ 0.36019656,  0.4598757 , -0.76666754, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.36163741,  0.45987812, -0.77763826, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.36246556,  0.4598732 , -0.72000152, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.36331969,  0.45985696, -0.73587692, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.36442015,  0.46002415, -0.73964107, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.36644372,  0.45956641, -0.69427419,

IndexError: list index out of range

: 

In [ ]:
# Emergency Camera Release
cap.release()
cv2.destroyAllWindows()